In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scienceplots
import networkx as nx
import ast

from nanonets import Simulation
from nanonets.utils import fft, cross_correlation
from matplotlib.animation import FuncAnimation
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

plt.style.use(["science","bright","grid"])
PATH = "/home/j/j_mens07/bagheera/data/2_funding_period/dynamic/AC/amp_freq/"

In [2]:
AMPLITUDE_LIST  = [0.02,0.03,0.04,0.05,0.06,0.07,0.08]
FREQ_LIST_MHZ   = [0.12,12.0,18.0,28.0,86.0]
N_PERIODS       = 100
N_SKIP          = 80
SAMPLE_P_PERIOD = 40
N_ELECTRODES    = 8
N_NP            = 9

topo = {"Nx": N_NP,"Ny": N_NP, "e_pos": [[(N_NP-1)//2, 0],[0, 0],[N_NP-1, 0],
               [0, (N_NP-1)//2],[N_NP-1, (N_NP-1)//2],[0, N_NP-1],
               [N_NP-1, N_NP-1],[(N_NP-1)//2, N_NP-1]],
            "electrode_type": ['constant']*8}
sim_class   = Simulation(topo)
G_obj       = sim_class.get_graph()
pos         = sim_class.get_positions()
pos         = {(key+8 if key >= 0 else (abs(key)-1)) : (val) for key, val in pos.items()}
E_CHARGE    = sim_class.ELE_CHARGE_A_C
CAP_MAT     = sim_class.get_capacitance_matrix()

### Electric Current

In [3]:
e_type  = 'constant'
dts     = {freq : 1/(40 * freq * 1e6) for freq in FREQ_LIST_MHZ}
N_steps = {freq : int(np.ceil((N_PERIODS / (freq * 1e6)) / dts[freq])) for freq in FREQ_LIST_MHZ}
time    = {freq : dts[freq]*np.arange(N_steps[freq]) for freq in FREQ_LIST_MHZ}
data    = {amp : {freq : pd.read_csv(f"{PATH}Nx=9_Ny=9_Ne=8_{freq:.3f}_{amp:.3f}_{e_type}.csv") for freq in FREQ_LIST_MHZ} for amp in AMPLITUDE_LIST}

In [5]:
amp = 0.02
n_p = 5

for freq in FREQ_LIST_MHZ:
    t = time[freq][:-1][:n_p*SAMPLE_P_PERIOD+1]*1e6
    x = data[amp][freq]['E0'].values[N_SKIP*SAMPLE_P_PERIOD:(N_SKIP+n_p)*SAMPLE_P_PERIOD+1]*1000
    y = data[amp][freq]['Observable'].values[N_SKIP*SAMPLE_P_PERIOD:(N_SKIP+n_p)*SAMPLE_P_PERIOD+1]*1e-6

    fig = plt.figure(dpi=200)
    ax1 = fig.add_subplot()
    ax1.plot(x, y)
    ax1.set_xlabel("$U$ [mV]")
    ax1.set_ylabel("$I$ [pA]")

    num_arrows      = 20
    arrow_indices   = np.linspace(0, len(x) - 2, num_arrows, dtype=int)

    x_pos = x[arrow_indices]
    y_pos = y[arrow_indices]
    x_dir = x[arrow_indices + 1] - x_pos
    y_dir = y[arrow_indices + 1] - y_pos

    ax1.quiver(x_pos, y_pos, x_dir, y_dir, scale_units='xy', angles='xy', scale=1.5, width=0.008, headwidth=5, zorder=10)

    fig.savefig(f"plots/svg/input_output/lissajous_{freq:.3f}_{amp:.3f}.svg", transparent=True)
    fig.savefig(f"plots/pdf/input_output/lissajous_{freq:.3f}_{amp:.3f}.pdf", transparent=False)
    fig.savefig(f"plots/png/input_output/lissajous_{freq:.3f}_{amp:.3f}.png", transparent=False)
    plt.close(fig)

In [6]:
amp = 0.02
n_p = 5
for freq in FREQ_LIST_MHZ:
    t       = time[freq][:-1][:-N_SKIP*SAMPLE_P_PERIOD]*1e6
    dt      = dts[freq]
    x       = data[amp][freq]['E0'].values[N_SKIP*SAMPLE_P_PERIOD:]*1000
    y       = data[amp][freq]['Observable'].values[N_SKIP*SAMPLE_P_PERIOD:]*1e-6
    y_e     = data[amp][freq]['Error'].values[N_SKIP*SAMPLE_P_PERIOD:]*1e-6
    xf, yf  = fft(y, dt=dt, use_hann=True)

    fig = plt.figure(dpi=200)
    ax1 = fig.add_subplot()
    ax1.plot(t, x)
    ax1.set_xlabel("$t$ [$\mu$s]", fontsize='large')
    ax1.set_ylabel("$U$ [mV]", fontsize='large')
    ax1.set_xlim(0, t[n_p*SAMPLE_P_PERIOD])

    fig.savefig(f"plots/svg/input_output/input_{freq:.3f}_{amp:.3f}.svg", transparent=True)
    fig.savefig(f"plots/pdf/input_output/input_{freq:.3f}_{amp:.3f}.pdf", transparent=False)
    fig.savefig(f"plots/png/input_output/input_{freq:.3f}_{amp:.3f}.png", transparent=False)
    plt.close(fig)

    fig = plt.figure(dpi=200)
    ax1 = fig.add_subplot()
    ax1.plot(t, y)
    ax1.fill_between(t, y-y_e, y+y_e, alpha=0.5, color='grey')
    ax1.set_xlabel("$t$ [$\mu$s]")
    ax1.set_ylabel("$I$ [pA]")
    ax1.set_xlim(0, t[n_p*SAMPLE_P_PERIOD])
    fig.savefig(f"plots/svg/input_output/output_{freq:.3f}_{amp:.3f}.svg", transparent=True)
    fig.savefig(f"plots/pdf/input_output/output_{freq:.3f}_{amp:.3f}.pdf", transparent=False)
    fig.savefig(f"plots/png/input_output/output_{freq:.3f}_{amp:.3f}.png", transparent=False)
    plt.close(fig)

    fig, ax = plt.subplots(dpi=200)
    for i in range(16):
        ax.vlines(i*freq, 1e-4, 1e1, color='black', ls='--', lw=.5)

    ax.plot(xf*1e-6, yf/np.max(yf))
    ax.set_yscale('log')
    ax.set_xlim(0.0, 10*freq)
    ax.set_ylim(1e-4, 1e1)
    ax.set_xlabel("$f$ [MHz]", fontsize='large')
    ax.set_ylabel("$FFT$", fontsize='large')
    ax.grid(False)

    fig.savefig(f"plots/svg/input_output/fft_{freq:.3f}_{amp:.3f}.svg", transparent=True)
    fig.savefig(f"plots/pdf/input_output/fft_{freq:.3f}_{amp:.3f}.pdf", transparent=False)
    fig.savefig(f"plots/png/input_output/fft_{freq:.3f}_{amp:.3f}.png", transparent=False)
    plt.close(fig)